<a href="https://colab.research.google.com/github/anjelammcgraw/Finetuning-Mistral-7B-Instruct-v0.2/blob/main/9_Fine_tuning_Mistral_7B_Instruct_for_Summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### ⚠ IMPORTANT ⚠

You will need at least 15GB of VRAM (GPU RAM) to run this notebook.


# Task Fine-tuning a GPT-style model using `peft`, `transformers` and `bitsandbytes`


# 🤝 Breakout #1

### Setting Up Dependencies


In [ ]:
pip install jupyter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.4/123.4 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.5/93.5 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 8.0 MB/s eta 0:00:00


In [ ]:
!pip install -qU bitsandbytes datasets accelerate loralib peft transformers trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 49.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 33.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 101.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 11.4 MB/s eta 0:00:00


In [ ]:
import torch
torch.cuda.is_available()

True

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import torch
import torch.nn as nn
import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM, BitsAndBytesConfig

## Task #1: Loading the Model


In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
)

In [ ]:
model_id = "mistralai/Mistral-7B-Instruct-v0.2"

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map='auto',
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [ ]:
tokenizer.pad_token = tokenizer.unk_token
tokenizer.padding_side = "right"

### Model Architecture


In [ ]:
print(model)

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )
   

In [ ]:
model.config

MistralConfig {
  "_name_or_path": "mistralai/Mistral-7B-Instruct-v0.2",
  "architectures": [
    "MistralForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 32768,
  "model_type": "mistral",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "quantization_config": {
    "_load_in_4bit": true,
    "_load_in_8bit": false,
    "bnb_4bit_compute_dtype": "float16",
    "bnb_4bit_quant_type": "nf4",
    "bnb_4bit_use_double_quant": false,
    "llm_int8_enable_fp32_cpu_offload": false,
    "llm_int8_has_fp16_weight": false,
    "llm_int8_skip_modules": null,
    "llm_int8_threshold": 6.0,
    "load_in_4bit": true,
    "load_in_8bit": false,
    "quant_method": "bitsandbytes"
  },
  "rms_norm_eps": 1e-05,
  "rope_theta": 1000000.0,
  "sliding_window": null,
  "tie_word_embeddings": false,
 

# 🤝 Breakout #2:

## Task #2: Data and Data Preparation



In [ ]:
from datasets import load_dataset

dataset_name = "samsum"
dataset = load_dataset(dataset_name)

Generating train split:   0%|          | 0/14732 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/819 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/818 [00:00<?, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

In [ ]:
dataset["test"] = dataset["test"].select(range(50))

In [ ]:
dataset["train"] = dataset["train"].select(range(1000))

In [ ]:
dataset["validation"] = dataset["validation"].select(range(50))

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 50
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 50
    })
})

In [ ]:
dataset["train"][0]

{'id': '13818513',
 'dialogue': "Amanda: I baked  cookies. Do you want some?\r\nJerry: Sure!\r\nAmanda: I'll bring you tomorrow :-)",
 'summary': 'Amanda baked cookies and will bring Jerry some tomorrow.'}

In [ ]:
def create_prompt(sample, include_response = True):
  """
  Parameters:
    - sample: dict representing row of dataset
    - include_response: bool

  Functionality:
    This function should build the Python str `full_prompt`.

    If `include_response` is true, it should include the summary -
    else it should not contain the summary (useful for prompting) and testing

  Returns:
    - full_prompt: str
  """

def create_prompt(sample, include_response=False):
    """
    Parameters:
    - sample: dict representing a row of the dataset
    - include_response: bool, determines whether to include the summary in the output

    Functionality:
    Constructs a formatted prompt that asks to provide a summary of a given dialogue.
    It formats the dialogue and optionally includes the summary based on the 'include_response' flag.

    Returns:
    - full_prompt: str, the constructed prompt following the specified format
    """

    # Extracting dialogue and summary from the sample
    dialogue = sample['dialogue']
    summary = sample['summary']

    # Preparing the dialogue by escaping newline characters for consistent display
    dialogue_formatted = dialogue.replace('\n', '\\n')  # Adjust based on your environment's newline character handling

    # Constructing the full prompt with instructional text and optional summary
    instructional_text = "<s>[INST]Provide a summary of the following text:\n\n[INPUT_TEXT_START]\n"
    text_end = "\n[INPUT_TEXT_END]\n\n[/INST]\n"
    summary_text = f"{summary}" if include_response else ""

    # Assemble the full prompt
    full_prompt = f"{instructional_text}{dialogue_formatted}{text_end}SUMMARY{summary_text}"

    return full_prompt
def some_function():
    # ...
    return full_prompt

In [ ]:
print(create_prompt(dataset["test"][1]))

<s>[INST]Provide a summary of the following text:

[INPUT_TEXT_START]
\nRob: Sure :)
[INPUT_TEXT_END]

[/INST]
SUMMARY


In [ ]:
def generate_response(prompt, model, tokenizer):
  """
  Parameters:
    - prompt: str representing formatted prompt
    - model: model object
    - tokenizer: tokenizer object

  Functionality:
    This will allow our model to generate a response to a prompt!

  Returns:
    - str response of the model
  """

  # convert str input into tokenized input
  encoded_input = tokenizer(prompt,  return_tensors="pt")

  # send the tokenized inputs to our GPU
  model_inputs = encoded_input.to('cuda')

  # generate response and set desired generation parameters
  generated_ids = model.generate(
      **model_inputs,
      max_new_tokens=256,
      do_sample=True,
      pad_token_id=tokenizer.eos_token_id
  )

  # decode output from tokenized output to str output
  decoded_output = tokenizer.batch_decode(generated_ids)

  # return only the generated response (not the prompt) as output
  return decoded_output[0].split("[/INST]")[-1]

In [ ]:
generate_response(create_prompt(dataset["test"][1], include_response=False),
                  model,
                  tokenizer)

'\nSUMMARY</s>'

In [ ]:
# Ground Truth Summary
dataset["test"][1]["summary"]

'Eric and Rob are going to watch a stand-up on youtube.'

In [ ]:
generate_response(create_prompt(dataset["test"][3], include_response=False),
                  model,
                  tokenizer)

'\nSUMMARY</s>'

In [ ]:
# Ground Truth Summary
dataset["test"][3]["summary"]

'Emma will be home soon and she will let Will know.'

### Required Post Processing



In [ ]:
from peft import prepare_model_for_kbit_training
model.config.use_cache = False
model = prepare_model_for_kbit_training(model)

## Task #3: Setting up PEFT LoRA


#### Helper Function to Print Parameter %age


In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
from peft import LoraConfig, get_peft_model

# set our rank (higher value is more memory/better performance)
lora_r = 16

# set our dropout (default value)
lora_dropout = 0.1

# rule of thumb: alpha should be (lora_r * 2)
lora_alpha = 32

# construct our LoraConfig with the above hyperparameters
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM"
)

In [ ]:
model = get_peft_model(
    model,
    peft_config
)

print_trainable_parameters(model)

trainable params: 6815744 || all params: 3758886912 || trainable%: 0.18132346515244138


In [ ]:
print(model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): Linear4bit(in_features=4096, out_features=1024, bia

#### ❓Question #3:

What modules (or groupings of layers) did we apply LoRA too - and how can we tell from the model summary?

#### Answer #3:
In the self_attn (self-attention) module of each MistralDecoderLayer, you can see lora.Linear4bit applied to both q_proj (query projection) and v_proj (value projection).The presence of lora_dropout, lora_A, and lora_B within these lora.Linear4bit instances confirms the application of LoRA.

## Task #4: Training the Model



### Setting up Training


In [ ]:
from transformers import TrainingArguments

args = TrainingArguments(
  output_dir = "mistral7binstruct_summarize",
  #num_train_epochs=5,
  max_steps = 50, # comment out this line if you want to train in epochs
  per_device_train_batch_size = 1,
  warmup_steps = 0.03,
  logging_steps=10,
  #evaluation_strategy="epoch",
  evaluation_strategy="steps",
  eval_steps=25, # comment out this line if you want to evaluate at the end of each epoch
  learning_rate=2e-4,
  lr_scheduler_type='constant',
)

#### ❓Question #4:

Describe what the following parameters are doing:

- `warmup_steps`: **specifies the number of steps for the learning rate warmup phase and gradually increases the learning rate from 0 (or a lower value) to the initial learning rate (0.3)**
- `learning_rate`: **parameter sets the initial size of the updates to the model weights and determines the step size at each iteration while moving toward a minimum of a loss function (2e-4)**
- `lr_scheduler_type`: **defines the learning rate scheduler type to use (constant)**



In [ ]:
from trl import SFTTrainer

max_seq_length = 2048

trainer = SFTTrainer(
  model=model,
  peft_config=peft_config,
  max_seq_length=max_seq_length,
  tokenizer=tokenizer,
  packing=True,
  formatting_func=create_prompt,
  args=args,
  train_dataset=dataset["train"],
  eval_dataset=dataset["validation"]
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/utils.py:434: UserWarning: The passed formatting_func has more than one argument. Usually that function should have a single argument `example` which corresponds to the dictionary returned by each element of the dataset. Make sure you know what you are doing.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed `packing=True` to the SFTTrainer, and you are training your model with `max_steps` strategy. The dataset will be iterated until the `max_steps` are reached.
  warnings.warn(


In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss
25,1.717200,1.492838
50,1.560300,1.426370


TrainOutput(global_step=50, training_loss=1.676695499420166, metrics={'train_runtime': 523.8143, 'train_samples_per_second': 0.095, 'train_steps_per_second': 0.095, 'total_flos': 4372977156096000.0, 'train_loss': 1.676695499420166, 'epoch': 0.46})

## Task #5: Share Your Model!


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
trainer.push_to_hub("mistral-7binstruct-summary-100s - with 100 steps")

adapter_model.safetensors:   0%|          | 0.00/27.3M [00:00<?, ?B/s]

events.out.tfevents.1709673411.679b6df80b60.3917.0:   0%|          | 0.00/6.97k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.92k [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/anjelammcgraw/mistral7binstruct_summarize/commit/e829d070b28eb82067d930d4bc41cc8cf66a483a', commit_message='mistral-7binstruct-summary-100s - with 100 steps', commit_description='', oid='e829d070b28eb82067d930d4bc41cc8cf66a483a', pr_url=None, pr_revision=None, pr_num=None)

### Compare Outputs


In [ ]:
merged_model = model.merge_and_unload()

/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/bnb.py:272: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


#### ❓Question #5:

What does the `merge_and_unload()` method do?

#### Answer:
The merge_and_unload() method is used to merge LoRA weights back into the main model, specifically for LoRA and PEFT trained models. This method merges the LoRa layers into the base model, allowing the base model to be used as a standalone model. It is essential for integrating LoRA weights back into the base model effectively.


In [ ]:
generate_response(create_prompt(dataset["test"][1], include_response=False),
                  merged_model,
                  tokenizer)

'\nSUMMARY</s>'

Let's look at the base model response:

> *Eric and Rob are having a conversation about a stand-up comedy performance by an unnamed comedian named "Machine." They find the performance hilarious, particularly a part involving a train. They discuss how Americans perceive Russians based on the comedy routine. Eric wonders if this is Machine\'s only stand-up performance, and Rob agrees to help him find more of Machine\'s comedy content on YouTube. They both express excitement and plan to watch the other performances together. They sign off with the phrase "talk to you later" (TTYL) before ending the conversation.</s>*

Now the fine-tuned response:

> *Eric and Rob watched a MACHINE comedy stand-up video and found it funny. Eric is especially impressed by the Russian accent in the video. He asks Rob if that was MACHINE'S only stand-up and if there are any others. Rob confirms that there are several MACHINE'S stand-up videos on Youtube. Eric and Rob agree to watch them together.</s>*



In [ ]:
print(dataset["test"][3]["dialogue"])

Will: hey babe, what do you want for dinner tonight?
Emma:  gah, don't even worry about it tonight
Will: what do you mean? everything ok?
Emma: not really, but it's ok, don't worry about cooking though, I'm not hungry
Will: Well what time will you be home?
Emma: soon, hopefully
Will: you sure? Maybe you want me to pick you up?
Emma: no no it's alright. I'll be home soon, i'll tell you when I get home. 
Will: Alright, love you. 
Emma: love you too. 


In [ ]:
generate_response(create_prompt(dataset["test"][3], include_response=False),
                  merged_model,
                  tokenizer)

'\nSUMMARY</s>'

In [ ]:
# Ground Truth Summary
dataset["test"][3]["summary"]

'Emma will be home soon and she will let Will know.'

Lets look at the base model response:

>  *Emma won't be home for dinner tonight, she has a problem, but she doesn't want Will to worry. She'll let him know when she's home. She'll be coming soon.</s>*

And the fine-tuned model:

> *Emma is not feeling well. She will be home soon. She doesn't want Will to cook anything for dinner.</s>*

